---
---

### Bu dosya içerisinde yazılan bütün kodların açıklamalarına ve detaylarına odaklanacağız.Nerde neden böyle bir şey yaptık.Hepsini daha detaylı inceleyeceğiz.

---
---

## Şu an odakta dosya:

### **perception_types.hpp**

İçindeki 3 temel yapı taşını birlikte netleştirelim:

* ImageFrame,

* DetectedObject

* PerceptionConfig

---

---

## İlk odaklanılacak kod bloğu ektedir :

```cpp
#ifndef PERCEPTION_TYPES_HPP
#define PERCEPTION_TYPES_HPP

#include <cstdint>
#include <string>
#include <vector>
#include <chrono>

#include <opencv2/core.hpp>

struct ImageFrame
{
    cv::Mat Image;
    std::uint64_t frame_id{0};

    std::chrono::steady_clock::time_point timestap{
        std::chrono::steady_clock::now()};

    std::string camera_name;

    ImageFrame() = default;

    ImageFrame(const cv::Mat &img, std::uint64_t id, const std::string &cam)
        : Image(img),
          frame_id(id),
          timestap(std::chrono::steady_clock::now()),
          camera_name(cam)
    {
    }
};


----
----

# 1) #ifndef PERCEPTION_TYPES_HPP – Bu ne anlama geliyor ? 

```cpp
#ifndef PERCEPTION_TYPES_HPP
#define PERCEPTION_TYPES_HPP
...
#endif // PERCEPTION_TYPES_HPP
```
* Bu yapı header guard (başlık koruması).

Amacı:

* Aynı .hpp dosyası birden fazla kez include edilirse derleyiciye “bu dosyayı içerik olarak sadece bir kez derle” demek.

* C++’ta aynı struct/class iki kez tanımlanırsa compile-time error alırsız (“redefinition” hatası).

### Çalışma mantığı:
```cpp
#ifndef PERCEPTION_TYPES_HPP
```

→ “Eğer PERCEPTION_TYPES_HPP makrosu daha önce tanımlanmadıysa buradaki kodu derle” demek.

İlk include’da:

* Makro tanımlı değil → içeri girer.

* #define PERCEPTION_TYPES_HPP ile makroyu tanımlar.

* Dosyanın geri kalanı derlenir.

İkinci include’da:

* Makro zaten tanımlı → #ifndef bloğuna girmez → dosya içeriği yok sayılır.

Sonuç:

* Dosyanın içeriği projede kaç yerde include edilirse edilsin, gerçek tanımlar sadece bir kez derlenir.

* Alternatif: #pragma once da aynı işi yapar ama header guard daha klasik ve taşınabilir.


----

# 2) std::uint64_t frame_id{0}; – Bu ne?

```cpp
    std::uint64_t frame_id{0};
```


### std::uint64_t nedir?

* std::uint64_t, cstdint header’ından gelen bir tip.

* Anlamı: işaretsiz 64 bit tamsayı (unsigned 64-bit integer).

**Aralık:**
>0 ile 2⁶⁴ − 1 arası (çok büyük).

### Neden kullanıyoruz?

**frame_id için:**

* Frame sayısı arttıkça taşma istemiyoruz.

* int kaç bit, platforma göre değişebiliyor; uint64_t kesin ve net.

* Loglama, ID verme, büyük kayıt sistemlerinde uzun süre çökmeden kullanabilirsin.

**Yani: frame_id için “büyük, taşmayacak, negatif olmayacak” bir sayaç tipi.**


### frame_id{0} ifadesindeki {0} nedir?

* Bu, in-class member initializer.

**frame_id{0};**
>→ Bu üye için varsayılan değer 0 olsun demek.

### Ne zaman kullanılır?

**Default constructor kullanıldığında veya herhangi bir constructor bu üyeyi init list’te açıkça başlatmazsa:**

* frame_id otomatik olarak 0 ile başlatılır.

### Neyi ifade ediyor?

**“Bu struct oluşturulduğunda, özel bir id verilmediyse, frame_id yi 0’dan başlat.”**

Bu hem hatayı azaltır hem de tanımsız değer (garbage) kullanmanı engeller.

----

# 3) std::chrono / steady_clock::time_point / timestamp

```cpp
    std::chrono::steady_clock::time_point timestap{
        std::chrono::steady_clock::now()};
```

### **std::chrono nedir?**

* C++’ın zaman ve süre ölçümü için sağladığı standart kütüphane (header: <chrono).

İçinde 3 temel kavram var:

* clock → saat (şu an ne zaman?)

* time_point → bir zaman noktası

* duration → iki zaman noktası arasındaki süre

### **std::chrono::steady_clock nedir?**

* Monoton ilerleyen, geriye gitmeyen saat tipi.

* Sistem saatine (kullanıcının tarih/saat ayarına) bağlı değil.

Ölçüm senaryolarında kullanılır:

* Latency

* FPS

* timeout vs.

### **Fark:**

* system_clock → gerçek dünya saati (tarih-saat), ama kullanıcı değiştirebilir.

* steady_clock → sadece ileri gider, ölçüm için güvenlidir.


### **std::chrono::steady_clock::time_point ne?**

* steady_clock için bir “zaman noktası” tipi.

* “Bu frame şu an ne zamandı?” sorusunun cevabını sakladığın tür.

Kodda:
```cpp
std::chrono::steady_clock::time_point timestamp{
    std::chrono::steady_clock::now()
};
```


Bu ne yapıyor?

timestamp üyesi için varsayılan değer olarak:

* struct oluşturulduğu anda steady_clock::now() sonucu atanıyor.

* Yani: ImageFrame yaratıldığı anın zamanını kaydediyor.


### **timestamp değişkeninin görevi ne?**

* Bu frame ne zaman oluşturuldu? bilgisini tutuyor.

#### Kullanım alanları:

**Latency hesaplama:**
```cpp
auto now = std::chrono::steady_clock::now();
auto dt = now - frame.timestamp; // duration
```


**Loglama:**

* “Bu frame ne kadar sürede işlendi?”

* Çok sensörlü sistemde senkronizasyon.

>Gerçek zamanlı perception’de zaman damgası kritik.
>Node’lar arasında veri “ne kadar yaşlandı, hangi sırada?” gibi sorular bununla çözülür.

### **Parantez içindeki ifade tam olarak neyi temsil ediyor?**

Şuradaki ifade:
```cpp
std::chrono::steady_clock::time_point timestamp{
    std::chrono::steady_clock::now()
};


std::chrono::steady_clock::now()
```

→ Şu anda steady_clock’a göre zaman noktasını döndürür.

* Bu, timestamp’in varsayılan değeridir.

**Yani struct yaratıldığı anda timestamp otomatik olarak “şimdi”ye ayarlanır.**

---

# 4) std::string camera_name – Neden var?
```cpp
std::string camera_name;
```


Amaç:

* Bu frame hangi kameradan geldi? bilgisini taşımak.

Örnek senaryo:

* Aracın 4 kamerası var: front, rear, left, right.

* Hepsi aynı PerceptionNode tarafından işleniyor olabilir.

* Logda / hata analizinde şunu bilmek istersin:

>“Bu tespit ön kameradan mı geldi, yan kameradan mı?

## Kullanımı:
```cpp
ImageFrame frame(img, 1, "front_camera");
```


Böylece:

### Logging:
```cpp
std::cout << "Camera: " << frame.camera_name << "\n";
```


### ROS2 / mesajlaşma:
```cpp
topic: /perception/front_camera/detections
```

**Tek cümle: Kamera kaynağını izlemek ve multi-camera sistemleri düzgün yönetmek için var.**

---

# 5) ImageFrame() default constructor – Neden var?
```cpp
ImageFrame() = default;
```


Bu satır şunu diyor:

**“Bana parametresiz bir constructor ver, ama derleyicinin oluşturduğu default versiyonu kullan.”**

Yani şu mümkün olsun:
```cpp
ImageFrame frame;
frame.image = someMat;
frame.frame_id = 42;
frame.camera_name = "front_camera";
```

### Neden ayrıca yazdık?

Çünkü aşağıda parametreli bir constructor tanımladık:
```cpp
ImageFrame(const cv::Mat& img,
           std::uint64_t id,
           const std::string& cam)
```


* Eğer bunu yazarsan, derleyici otomatik default ctor üretmeyi bırakabiliyor bazı durumlarda.

* Biz “hem parametreli constructor olsun, hem de default olsun” istiyoruz.

O yüzden:

**= default; ile “default constructor’ı da aktif bırak” demiş oluyoruz.**


----
# 6) ImageFrame(const cv::Mat &img, std::uint64_t id, const std::string &cam) – Amacı ne?



```cpp
ImageFrame(const cv::Mat& img,
           std::uint64_t id,
           const std::string& cam)
    : image(img)
    , frame_id(id)
    , timestamp(std::chrono::steady_clock::now())
    , camera_name(cam)
{}
```
Bu bir parametreli constructor. Amacı:

**Tek satırda tam dolu bir ImageFrame oluşturmak.**

Kullanım:
```cpp
cv::Mat img = cv::imread("test.jpg");
ImageFrame frame(img, 1, "front_camera");
```


## Bu satırda:

* image → img ile dolduruluyor.

* frame_id → 1 oluyor.

* timestamp → o an now() ile atanıyor.

* camera_name → "front_camera" oluyor.

### Neden önemli?

* Kodun daha temiz ve kısa olması için.

Her seferinde:
```cpp
ImageFrame frame;
frame.image = img;
frame.frame_id = 1;
frame.camera_name = "front_camera";
```
yazmak yerine tek satır kullanırsın.

>Ayrıca tutarlılık sağlar: constructor içinde timestamp’i otomatik set ettiği için ImageFrame her zaman “tarihli” olur, unutmazsın.


--- 
# 7) Genel olarak ImageFrame struct’ının amacı nedir?

### Kısa cevap:

**ImageFrame, bir kamera karesini tek bir veri paketi olarak temsil eder:
Görüntü + kimlik + zaman + kaynak kamera.**


### Daha net açalım:

Bu struct:

* perception modülüne input tipi olarak giriyor.

**PerceptionNode::process(const ImageFrame& frame) dediğinde:**

* Model için gerekli piksel bilgisi image içinden geliyor.

* Log/latency için gerekli zaman timestamp içinden geliyor.

* Hangi kameranın çıktısı olduğunu camera_name içinden biliyorsun.

* Frame sırası / id’si frame_id ile takip ediliyor.

**Sadece cv::Mat gönderseydin:**

* Zaman, id, kamera kaynak bilgisi kaybolurdu.

* Bunları global değişkenle veya yan yollardan taşımak zorunda kalırdık.

* Kod pisleşirdi.

**Bu struct ile:**

* Tek parametreyle her şeyi taşıyoruz.

* Fonksiyon imzaları temiz:
```cpp
std::vector<DetectedObject> process(const ImageFrame& frame);
```


----
----
---

## Yukarıda anlatılanlar ilk struct yapı olan **ImageFrame** ile alakalıydı.Şimdi odaklanacağımız kısım yine **perception_types** içerisinde bulunan bir diğer struct yapı: **DetectedObject**

---
---
---


# **DetectedObject**

```cpp
struct DetectedObject
{
    int class_id{-1};
    std::string class_name;
    float score{0.0f};

    float x{0.0f};
    float y{0.0f};
    float width{0.0f};
    float heigth{0.0f};

    int track_id{-1};
};

## Tek tek açıklaya açıklaya gidelim.Genel olarak bu anlatımı 3 parçaya bölelim:

* Bu struct nasıl bir yapı, amacı ne?

* İçindeki her alan tek tek ne anlama geliyor?

* Neden {} ve 0.0f / -1 gibi default değerler kullandık?

------

# 1) Bu struct ne tür bir yapı? Amacı ne?

* Bu struct, tek bir algılama sonucunu (tek bir nesneyi) temsil ediyor.

### Model ne yapıyor?
Görüntüye bakıyor ve diyor ki:

* Şurada bir sınıf buldum (örneğin car),

* Şu bölgededir (bbox: x, y, width, height),

* Bu tespitten şu kadar eminim (score).

**Biz bu bilgiyi ham tensor olarak tutmak istemiyoruz (ör. [x, y, w, h, obj_score, class_scores…]).
Bunu daha insan okunur ve yüksek seviyeli bir forma sokuyoruz.**

>İşte bu formun adı: DetectedObject.

* Yani DetectedObject, “modelin tek bir tespit ettiği nesne”dir.
Dış dünyaya veri verirken hep bu struct ile konuşacağız.

Bir frame’de:

* 0 tane DetectedObject olabilir,

* 5 tane olabilir,

* 20 tane olabilir.

Hepsi için :
```cpp
std::vector<DetectedObject> 
```

döneceğiz.

---

# 2) Alanları tek tek parçalayalım

## a) int class_id{-1};

* Modelin verdiği sınıf index’i.

Örneğin:

* 0 → person

* 1 → car

* 2 → truck

### -1 default değer:

* “Henüz set edilmedi / geçersiz” anlamına gelir.

* Bu ID, çoğu zaman PerceptionConfig::class_names ile birleşir:

```cpp
obj.class_name = config.class_names[obj.class_id];
```

## b) std::string class_name;

**İnsan okunabilir sınıf ismi:**

* "person", "car", "dog", "traffic_light" gibi.

Bunu tutmamızın sebebi:

* Loglarda, overlay yazılarda, JSON çıktılarda, debug ekranlarında
direkt metin olarak görmek istememiz.

* class_id her zaman anlamlı değil; "2" yerine "car" görmek istiyoruz.

* Default değer vermedik; C++ std::string’i zaten boş string ("") ile başlatıyor.

## c) float score{0.0f};

* Modelin bu nesne için verdiği güven skoru.

* Aralık tipik olarak [0.0, 1.0].

* 0.95 → çok emin

* 0.30 → sınırda

Postprocess’te score_threshold ile kıyaslanır:
```cpp
if (obj.score < config.score_threshold) discard;
```


### 0.0f neden?

>0.0f → float literali.
* Tek başına 0.0 yazarsan double olur, C++ implicit cast yapar ama biz tipi net tutuyoruz.

Default değerin 0 olması:

* “Set edilmemiş durumda bu tespit değersizdir.” hissini verir.

* Yanlışlıkla score’a bakarsan “aa 0, demek set edilmemiş/düşük” diyebilirsin.

## d) Bbox alanları: x, y, width, height
```cpp
float x{0.0f};
float y{0.0f};
float width{0.0f};
float height{0.0f};
```


Bunlar piksel cinsinden sınır kutusunu temsil ediyor:

* (x, y) → sol–üst köşe (top-left) koordinatı

- width → kutunun genişliği

- height → kutunun yüksekliği

Örneğin:

**Görüntü: 1920x1080**

**Bbox: x = 480, y = 270, width = 960, height = 540**

**Bu, görüntünün ortasında genişçe bir kutu demek.**

### Neden float?

* Model çıktıları genellikle float (PyTorch tensor’ler).

* Fractional koordinatlar olabilir (örneğin scale/resize sonrası).

### Neden default 0.0f?

Varsayılanda “0, 0, 0, 0” bir bbox:

* Üzerinde işlem yapılmamış

* “Anonim/dummy” değer.

**Özellikle Uygulama 1’de dummy detection oluştururken anlamlı bir bbox atıyoruz; ama default değerler güvenli bir başlangıç noktası sağlıyor.**

## e) int track_id{-1};

* Tracking (nesne takibi) için kullanılan ID.

Amaç:

* Bir nesneyi farklı framelerde aynı nesne olarak tanımak.

Örneğin:

* Frame 1’de car → track_id = 7

* Frame 2’de aynı car → track_id = 7

### Default -1:

* “Bu nesne herhangi bir takip algoritması tarafından ilişkilendirilmemiş.”
anlamına gelir.
----


# 3) Neden çoğunda { ... } ve 0.0f / -1 kullandık?

## **a) Süslü parantez {} kullanımı**

### Şu yazım:
```cpp
int class_id{-1};
float score{0.0f};
float x{0.0f};
...
```
* → Buna in-class member initializer diyoruz.


#### Anlamı:

* Bu üyeler için varsayılan değer belirliyorsun.

* Her constructor çağrıldığında, eğer init list'te açıkça başka değer verilmemişse, bu değerle başlatılıyor.

### Örnek:
```cpp
DetectedObject obj;  // default ctor kullanımı
```
#### Bu durumda:

* class_id → -1

* score → 0.0f

* x, y, width, height → 0.0f

* track_id → -1

Böylece:

* Çöplük (uninitialized) değer olmaz.

- Debug ederken her üyenin “başlangıçta ne olduğunu” bilirsin.


## **b) Neden 0.0f ve -1 gibi değerler seçtik?**

Bu değerler bilinçli:

* -1 → genelde “geçersiz / set edilmemiş / yok” anlamında kullanılır.

* class_id = -1 → model hiçbir şey set etmemiş.

* track_id = -1 → takip algoritması henüz bir ID vermemiş.

0.0f → score ve bbox için:

* “Henüz gerçek bir tespit yapılmamış, neutral başlangıç.”

* Bbox = (0,0,0,0) ise genelde tespit anlamına gelmez.

### Bu default değerleri seçmenin amacı:

#### Güvenli başlangıç:
* Uninitialized random çöp değeri yerine, tahmin edilebilir bir hal.

#### Hata yakalamayı kolaylaştırma:
* class_id == -1 gördüğünde “buraya hiç değer atanmamış” diyebilirsin.


## **c) Neden = 0, = -1 değil de {0.0f}, {-1}?**

İkisi de bu durumda çalışır ama {}:

* C++11’in “uniform initialization” tarzı,

* Narrowing conversion’lara (örneğin double → int) karşı daha hassas,

* Yeni kod stilinde daha çok tercih ediliyor.


>type name{value};
Modern C++’ta bir üye için varsayılan değer vermenin tavsiye edilen yoludur.

---
# 4) Toparlarsak – DetectedObject’in net tanımı

* DetectedObject yapısı, algılama modelinin tek bir tespitini temsil eder.

İçinde:

* class_id ve class_name ile nesnenin türü,

* score ile modelin güven seviyesi,

* x, y, width, height ile nesnenin görüntü üzerindeki konumu,

* track_id ile zaman içindeki takibi

tutulur.

**Varsayılan değerler (-1, 0.0f) sayesinde bu struct, oluşturulduğu anda güvenli bir başlangıç durumuna sahiptir; henüz set edilmemiş alanlar rahatça ayırt edilebilir.**

----
----
----

## Şimdi ise son yapıdayız.Artık bu uygulamamın son durağı: 

### **PerceptionConfig**

----
----
----

# **PerceptionConfig**

```cpp
struct PerceptionConfig
{
    std::string model_path;
    std::string device{"cpu"};

    int input_width{640};
    int input_height{640};

    std::vector<float> mean{0.0f, 0.0f, 0.0f};
    std::vector<float> std{1.0f, 1.0f, 1.0f};

    float score_threshold{0.25f};
    float nms_iou_threshold{0.45f};

    std::vector<std::string> class_names;
};
```


Bunu üç parçaya bölelim:

* Bu struct’ın genel amacı ne?

- İçindeki her alan ne işe yarıyor, neden bu değerler seçildi?

- Neden böyle bir struct ile çalışıyoruz (avantajı ne)?

---

# 1) PerceptionConfig nedir, amacı ne?

* Bu struct; PerceptionNode’u ayağa kaldırırken ona verdiğimiz konfigürasyon paketidir.

Yani:
```cpp
PerceptionConfig config;
// alanları dolduruyorsun...
PerceptionNode node(config);
```


Bu şu demek:

**“Bu node nasıl çalışsın, hangi modeli kullansın, hangi cihazda çalışsın, input boyutu ne olsun, hangi mean/std ile normalize etsin, hangi skorun altını çöpe atsın, sınıf isimleri ne olsun?”**
gibi tüm ayarları tek bir struct içinde topluyoruz.

Node’un içine onlarca parametre saçmak yerine, tek bir config nesnesi veriyoruz.

----
# 2) Alanları tek tek inceleyelim

## a) std::string model_path;
```cpp
std::string model_path;
````


* TorchScript modelinin dosya yolu.

>Ör: "../models/yolov8s.torchscript.pt"

>veya: "best_model.pt"

PerceptionNode constructor’ında şuna bakıyoruz:

* Eğer model_path boş değilse:
```cpp
loadModel(model_path);
```

Boşsa:

* Uygulama 1’deki gibi “model yüklenmeyecek, dummy moddayız” diye davranıyoruz.

Amacı:

* Node ile model dosyasını sıkı sıkıya bağlamak yerine, yolu dışarıdan belirtmek.

## b) std::string device{"cpu"};
```cpp
std::string device{"cpu"};
```


Node’un hangi cihazda çalışacağını belirler:

* "cpu"

- "cuda" (veya "cuda:0" tarzı)

Constructor’da kullandığımız mantık:
```cpp
if (config_.device == "cuda" && torch::cuda::is_available())
    device_ = torch::kCUDA;
else
    device_ = torch::kCPU;
```


Yani:

* "cuda" deyip GPU varsa → CUDA’da çalışır.

* GPU yoksa veya "cpu" dersen → CPU’da çalışır.

Default "cpu":

- Varsayılanı güvenli tutuyoruz.

- GPU yoksa çökmez, CPU’da devam eder.


Amacı:

**Config seviyesinde “bu node CPU’da mı, GPU’da mı koşacak?” kararını vermek.**

## c) int input_width{640}; int input_height{640};
```cpp
int input_width{640};
int input_height{640};
```


* Modelin beklediği giriş boyutu.

* YOLO için klasik 640x640.

preprocess() içinde şuna benzer bir kullanım olacak:
```cpp
cv::Mat resized;
cv::resize(frame.image, resized, cv::Size(config_.input_width, config_.input_height));
```


### Neden önemli?

* TorchScript modelin eğitilirken hangi boyutta input aldıysa, inference’ta da ona göre çalışması gerekir.

- YOLO gibi modeller genellikle sabit input size kullanır.

#### Default 640x640:

* Çok genel, pratik bir seçim.

* Sonradan başka model için 320x320, 1024x1024 gibi değiştirebilirsin.

## d) std::vector<float> mean{0.0f, 0.0f, 0.0f};
```cpp
std::vector<float> mean{0.0f, 0.0f, 0.0f};
```


* Normalize için kullanılacak kanal başına ortalama (mean) değerleri.

Tipik kullanım (PyTorch tarzı):
```cpp
tensor = (tensor - mean) / std;
```


Örneğin ImageNet pretrained bir model için genelde:
```python
mean = {0.485f, 0.456f, 0.406f};
std  = {0.229f, 0.224f, 0.225f};
```


* Biz şimdilik defaultu nötr tuttuk: {0,0,0} (hiç offset yok).

Neden vector?
* 
3 kanallı (RGB) input için 3 değer gerekiyor.

* Daha sonra 1 kanal / 4 kanal gibi durumlara genişletebilirsin.

Amacı:
**Preprocess aşamasında görüntüyü modelin beklediği dağılıma çekmek.**


## e) std::vector<float> std{1.0f, 1.0f, 1.0f};
```cpp
std::vector<float> std{1.0f, 1.0f, 1.0f};
```


* Normalize için standart sapma (std) değerleri.

Formül:
```bash
normalized = (x - mean) / std;
```


Default {1,1,1}:

* Bölme etkisiz: (x - mean) / 1 = x - mean.

* Yani sadece mean kullanılmış oluyor.

#### Gerçek model için:

* PyTorch modellerinde genelde ImageNet değerleri set edilir.

Amacı:
**Preprocess’te kanal bazlı normalizasyonu parametreleştirmek**


## f) float score_threshold{0.25f};
```cpp
float score_threshold{0.25f};
```


* postprocess() aşamasında kullanılan minimum skor eşiği.

Örneğin YOLO’da:
```python
if (score < config_.score_threshold)
    continue; // atla
```


Default 0.25:

* YOLO ailesi için gayet makul bir başlangıç değeri.

**Çok düşük seçersen:**

* Gürültü artar, çok fazla düşük güvenli kutu gelir.

**Çok yüksek seçersen:**

* Gerçek nesnelerden bazılarını kaçırırsın.

Amacı:
**Zayıf ve önemsiz tespitleri daha postprocess’in başında elemek.**

## g) float nms_iou_threshold{0.45f};

```cpp

float nms_iou_threshold{0.45f};

```

* Non-Maximum Suppression (NMS) için kullanılan IoU eşiği.

* Aynı nesneye ait, birbirine çok benzeyen kutuları temizlemek için.

İşleyiş kabaca:

* Aynı sınıfta ve yakın pozisyonda iki bbox düşün:

* IoU > 0.45 ise bunlardan birini at, sadece en yüksek skorlu olanı tut.

Default 0.45:

* Yine YOLO default’larına çok yakın.

* 0.5 civarındaki değerler genelde dengeli sonuç verir.

Amacı:
**Çakışan kutuları temizleyip daha temiz bir tespit listesi oluşturmak**

## h) std::vector<std::string> class_names;
```cpp
std::vector<std::string> class_names;
```


>Index → sınıf adı mapping’i.

Örnek doldurma:
```cpp
config.class_names = {
    "person",
    "bicycle",
    "car",
    "motorbike",
    ...
};
```


Sonra postprocess() içinde:
```cpp
obj.class_name = config_.class_names[obj.class_id];
```


### Neden bu kadar önemli?

* Model genelde sadece class id (int) verir.

* UI, log, JSON vs. için metin lazım.

* Sınıf sayısı değişince sadece bu listeyi güncelleyerek sistemi uyarlayabilirsin.

Amacı:
**class_id → class_name dönüşümünü tamamen config seviyesine taşımak.**

---

# 3) Neden böyle bir struct ile çalışıyoruz?

* Burası işin mimari tarafı.


## a) Node’u “parametre çöplüğünden” kurtarmak için

Şöyle bir constructor hayal et:

```cpp
PerceptionNode(const std::string& model_path,
               const std::string& device,
               int input_width,
               int input_height,
               const std::vector<float>& mean,
               const std::vector<float>& std,
               float score_threshold,
               float nms_iou_threshold,
               const std::vector<std::string>& class_names);
```

#### Bu:

* Okunması zor,

* Çağırması zor,

* Bakımı felaket.

#### Bunun yerine:

```cpp
PerceptionConfig config;
// alanları doldur
PerceptionNode node(config);
```

* Çok daha temiz.


## b) Çalışma şeklini koddan bağımsız değiştirebilmek için

Model veya ayar değiştirmek için:

* Kodun içine girip sabitleri (magic numbers) değiştirmek yerine,

* Sadece PerceptionConfig’i değiştirmek yeterli.

Örneğin:

* CPU’dan CUDA’ya geçmek:
```cpp
config.device = "cuda";
```

Input boyutunu 320x320 yapmak:
```cpp
config.input_width = 320;
config.input_height = 320;
```


Sınıfları değiştirmek:
```cpp
config.class_names = {"helmet", "vest", "person"};
```


* Node’un iç mantığı değişmiyor; sadece ayarlarla oynuyorsun.

## c) Aynı PerceptionNode’u farklı projelerde kullanabilmek için

* Aynı sınıfı, farklı modellerle, farklı input size’larla, farklı class set’leriyle kullanmak istiyorsun.

Bunu sağlamanın tek düzgün yolu:
**Davranışı config ile kontrol etmek.**